In [2]:
import csv
import os
import traceback
import numpy as np
import tqdm
import pandas as pd
import multiprocessing as mp
import gc
from datetime import timedelta, date
import uuid

import rsa_data_summary as rd
import rsa_data_wim as wim
import rsa_headers as rh
import config
import queries as q
import tools

pd.options.mode.chained_assignment = None
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [3]:
# individual tables for type 30 and 60 test
# files = tools.getfiles(r'C:\FTP\Syntell\SMEC RSA Files_GP PRM Sites_Dec21toFeb22')
# file = r"C:\FTP\Syntell\0087_20220331.RSA"
t10_file = r"C:\FTP\Syntell\SMEC RSV Files_GP PRM Sites_Dec21toFeb22_individuals\0006-20211231.RSV"
# file = r"C:\Users\MB2705851\OneDrive - Surbana Jurong Private Limited\Manuals & Guidelines\Traffic\example data\type21_60_70.RSA"
type60_file = r"C:\Users\MB2705851\OneDrive - Surbana Jurong Private Limited\Manuals & Guidelines\Traffic\example data\type21_60_70.RSA"

In [4]:
# electronic_count_data_type_21_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_data_type_21 limit 1",config.ENGINE).columns)
# electronic_count_data_type_30_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_data_type_30 limit 1",config.ENGINE).columns)
electronic_count_data_type_60_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_data_type_60 limit 1",config.ENGINE).columns)
# electronic_count_data_type_70_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_data_type_70 limit 1",config.ENGINE).columns)
# header_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_header limit 1",config.ENGINE).columns)

In [5]:
df = tools.to_df(type60_file)

In [4]:
header_df = rh.Headers.get_head(df)

In [5]:
def select_classification_scheme(classification_scheme):
    if int(classification_scheme) == 8:
        vc_df = pd.read_sql_query(q.SELECT_CLASSIFICAITON_SCHEME_8, config.ENGINE)
    elif int(classification_scheme) == 1:
        vc_df = pd.read_sql_query(q.SELECT_CLASSIFICAITON_SCHEME_1, config.ENGINE)
    elif int(classification_scheme) == 5:
        vc_df = pd.read_sql_query(q.SELECT_CLASSIFICAITON_SCHEME_5, config.ENGINE)
    elif int(classification_scheme) == 9:
        vc_df = pd.read_sql_query(q.SELECT_CLASSIFICAITON_SCHEME_9, config.ENGINE)
    else:
        vc_df = None
    return vc_df
    

In [6]:
data = df.loc[(df[0] == "60") & (df[1].isin(["0", "1", "2", "3", "4"]))].dropna(
    axis=1, how="all"
).reset_index(drop=True)
dfh = pd.DataFrame(
    df.loc[
        (df[0].isin(["60"]))
        & (~df[1].isin(["0", "1", "2", "3", "4"]))
    ]
).dropna(axis=1, how="all").drop_duplicates().reset_index(drop=True).copy()
dfh2 = pd.DataFrame(df.loc[(df[0].isin(["S0", "L1"]))]).dropna(
    axis=1, how="all"
).reset_index(drop=True).copy()

In [7]:
header = pd.DataFrame(
    df.loc[
        (df[0].isin(["H0", "S0", "I0", "S1", "D0", "D1", "D3", "L0", "L1", "L2", "L3","L4" ,"L5","L6","L7","L8","L9","L10","L11","L12"]))
        | (
            (df[0].isin(["21", "70", "30", "13", "60"]))
            & (~df[1].isin(["0", "1", "2", "3", "4"]))
        )
        | (
            (df[0].isin(["10"]))
            & (df[1].isin(["1", "8", "5", "9", "01", "08", "05", "09"]))
        )
    ]
).dropna(axis=1, how="all")

In [247]:
lanes_df = header.loc[
    (header[0].isin(["L0", "L1", "L2", "L3","L4" ,"L5","L6","L7","L8","L9","L10","L11","L12"]))
].dropna(axis=1).reset_index(drop=True).drop_duplicates()

c:\Users\MB2705851\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\frame.py:6242: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  diff = Index(subset).difference(self.columns)


In [8]:
def process_dates_and_times(df: pd.DataFrame, date_length: int, time_length: int, duration_min: int):
    df[3] = df[3].astype(str)
    df[3].loc[df[3].str[:2] == '24'] = ('0').zfill(7)

    if date_length == 6:
        df[2] = str(date.today())[:2] + df[2]
        df[2] = df[2].apply(lambda x: pd.to_datetime(x, format="%Y%m%d").date() + timedelta(days=1)
            if x[3] == '2400000' else pd.to_datetime(x, format="%Y%m%d").date())
    elif date_length == 8:
        df[2] = df[2].apply(lambda x: pd.to_datetime(x, format="%Y%m%d").date() + timedelta(days=1)
            if x[3] == '2400000' else pd.to_datetime(x, format="%Y%m%d").date())
    else:
        raise Exception("DATA Date length abnormal")

    if time_length < 7:
        df[3] = df[3].str.pad(width=7,side='right',fillchar="0")
        df[3] = df[3].apply(lambda x: pd.to_datetime(x, format="%H%M%S%f").time())
    else:
        df[3] = df[3].apply(lambda x: pd.to_datetime(x, format="%H%M%S%f").time())

    if (df[2].astype(str)+df[3].astype(str)).map(len).isin([20]).all():
        df['start_datetime'] = pd.to_datetime(df[2].astype(str)+df[3].astype(str), 
            format='%Y-%m-%d%H:%M:%S.%f') - timedelta(minutes=duration_min)
    else:
        df['start_datetime'] = pd.to_datetime((df[2].astype(str)+df[3].astype(str))[:18], 
            format='%Y-%m-%d%H:%M:%S') - timedelta(minutes=duration_min)
    return df

In [33]:
dfh.reset_index(drop=True)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,60,15,15,20,0400,0500,0600,0700,0800,0900,1000,1100,1200,1250,1300,1400,1500,1600,1700,1800,1900,2000,2100


In [9]:
#  FOR TYPE 60
if data.empty:
    pass
else:
    dir_1 = dfh2.iloc[1:,2].astype(int).min()
    dir_2 = dfh2.iloc[1:,2].astype(int).max()

    number_of_data_records = dfh.iloc[0,3]

    if data[1].isin(["0", "1", "2", "3", "4"]).any():
        ddf = data.iloc[:, 1:].reset_index(drop=True)
        ddf = pd.DataFrame(ddf).dropna(axis=1, how="all").reset_index(drop=True)

        site_id = dfh2.iloc[1,1]
        time_length = len(ddf[2][0])
        date_length = len(ddf[2][0])
        duration_min = int(ddf[4][0])
        ddf[4] = ddf[4].astype(int)
        ddf[5] = ddf[5].astype(int)
        max_lanes = ddf[5].max()

        ddf = process_dates_and_times(ddf, date_length, time_length, duration_min)

        ddf = ddf.apply(pd.to_numeric, errors='ignore')

        ddf.columns = ddf.columns.astype(str)

    else:
        pass
    # return df3

In [1]:
ddf.head()

NameError: name 'ddf' is not defined

In [43]:
df3 = pd.DataFrame(columns=['edit_code', 'start_datetime', 'end_date', 'end_time', 
'duration_of_summary', 'lane_number', 'bin_number', 'number_of_vehicles', 'bin_boundary_length_cm', 'direction', 'compas_heading'])
for lane_no in range(max_lanes):
    for i in range(1,int(number_of_data_records)):
        i=i+5
        join_to_df3 = ddf.loc[ddf['5'] == lane_no, ['1', 'start_datetime','2', '3', '4', '5', str(i), str(i), dfh.iloc[0:i+3], 'direction', 'compas_heading']]
        join_to_df3['class_number'] = i-5
        join_to_df3.rename(columns={'1':"edit_code",'2':"end_date",'3':"end_time",'4':"duration_of_summary",'5':'lane_number', str(i): 'number_of_vehicles'}, inplace=True)
        # df3 = pd.concat([df3,join_to_df3],keys=['start_datetime','lane_number','number_of_vehicles','class_number'],ignore_index=True, axis=0)
        break
df3['site_id'] = site_id
df3['year'] = int(df3['start_datetime'][0].year)
print(join_to_df3)


c:\Users\MB2705851\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\common.py:241: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


TypeError: unhashable type: 'DataFrame'

In [5]:
h_test = rh.Headers.headers(header_df, t10_file)

In [6]:
h_test

,start_datetime,end_datetime,site_id,document_url,header_id,number_of_lanes,station_name,speedbin1,speedbin2,speedbin3,speedbin4,speedbin5,speedbin6,speedbin7,speedbin8,speedbin9,dir1_id,dir2_id
0,2021-12-01,2021-12-31,0006,C:\FTP\Syntell\SMEC RSV Files_GP PRM Sites_Dec...,3462e40d-13a6-4691-975b-6a7ae5909655,4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,4
